In [1]:
## IPNI checker
import psycopg2 as pg
import genlist_api
import re
import csv
g = genlist_api.Genlist()
# postgresql database name
DSN = 'dbname=nvdimp user=psilotum'
conn = pg.connect(DSN)

In [9]:
df = []
with open('../../workspace/for_ipni.csv', 'r') as f:
    csvreader = csv.reader(f, delimiter='\t', quotechar='"')
    for row in csvreader:
        df.append(row)

In [36]:
results = []
for r in range(1,len(df)):
    myID = df[r][4]
    tbcName = df[r][2].replace("'","''")
    #if tbcName
    QUERY_IPNI_LOCAL = '''
    SELECT "taxonID", ipni_url, "scientificName" 
    FROM nomenclature.ipni
    WHERE "scientificName" like '%s%%'
    ''' % tbcName
    with conn:
        with conn.cursor() as curs:
            curs.execute(QUERY_IPNI_LOCAL)
            ipniResults = curs.fetchall()

    if len(ipniResults) == 1:
        ipniFullname = ipniResults[0][2]
        # 插入 postgresql 時要 escape '
        ipniTaxonID = ipniResults[0][0].replace("'", "''")
        ipniName = g.fmtname(ipniResults[0][2], doformat=False)[0]
        UPDATE_RL_IPNI = '''
        UPDATE twredlist2017
        SET ipni_taxonid = '%s',
            ipni_fullname = '%s',
            ipni_name = '%s'
        WHERE id = %s
        ''' % (ipniTaxonID, ipniFullname, ipniName, myID)
        print([myID, tbcName, ipniTaxonID, ipniFullname])
    else:
        results.append([myID, tbcName, ipniResults])

['4687', 'Veronicastrum loshanense', 'urn:lsid:ipni.org:names:77093130-1', 'Veronicastrum loshanense Tien T.Chen & F.S.Chou']
['5161', 'Viola nagasawai', 'urn:lsid:ipni.org:names:868748-1', 'Viola nagasawai Makino & Hayata']
['4532', 'Zanthoxylum integrifoliolum', 'urn:lsid:ipni.org:names:775789-1', 'Zanthoxylum integrifoliolum Merr.']
['5232', 'Zingiber kawagoii', 'urn:lsid:ipni.org:names:873051-1', 'Zingiber kawagoii Hayata']
['74', 'Adiantum ciliatum', 'urn:lsid:ipni.org:names:17009400-1', 'Adiantum ciliatum Blume']
['4004', 'Arthromeris lehmanni', 'urn:lsid:ipni.org:names:17365720-1', 'Arthromeris lehmannii Ching']
['410', 'Athyrium palustre', 'urn:lsid:ipni.org:names:17430340-1', 'Athyrium palustre Seriz.']
['832', 'Blumea hieracifolia', 'urn:lsid:ipni.org:names:55227-3', 'Blumea hieracifolia  var. holosericea (DC.) Benth.']
['1781', 'Castanopsis fabri', 'urn:lsid:ipni.org:names:295390-1', 'Castanopsis fabri Hance']
['731', 'Chenopodium formosanum', 'urn:lsid:ipni.org:names:164939

In [18]:
tbcName = 'Acalypha matuda'

In [27]:
g.fmtname(ipniResults[0][2], doformat=False)[0]

'Acalypha matudae'

In [30]:
ipniResults[0][0]

'urn:lsid:ipni.org:names:1377-2'